In [ ]:
import pandas as pd
from Servicio import Service
from sklearn.preprocessing import scale
import sqlalchemy as sa
s=Service()

In [2]:
import pandas as pd
import numpy  as np
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from wordcloud import WordCloud
import spacy  # For preprocessing
import nltk
import logging  # Setting up the loggings to monitor gensim
from time import time  # To time our operations
import re  # For preprocessing


In [4]:
sql="select d.id_documento,d.resumen,gr.grupo from documento d  join documentos_grupo_def gr on gr.id_documento=d.id_documento join documentos_doc2_vec dv on dv.id_documento =d.id_documento order by grupo;"
df=pd.read_sql(sa.text(sql),s.dao.engine)
s.dao.engine.dispose()
df

,id_documento,resumen,grupo
0,4249,octubre emanada honorable consejo directivo un...,0
1,21,juridico efectos codigo penal practica judicia...,0
2,4610,determinan parámetros ejercer interventores su...,0
3,2258,isabel goyes moreno universidad nariño investi...,0
4,2861,ciudad pasto francisco edmundo obando universi...,0
...,...,...,...
8071,2517,marco teorico acondicionamiento peces transpor...,31
8072,7051,evaluación crecimiento poblacional diatomea th...,31
8073,2433,efecto dióxido carbono concentraciones producc...,31
8074,7066,elaboración galantina remplazando zanahoria da...,31


In [7]:
import spacy
import spacy.cli
from nltk import word_tokenize
from string import punctuation
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

spacy.cli.download("es_core_news_sm")
nlp = spacy.load('es_core_news_sm')

stemmer = SnowballStemmer('spanish')
nltk.download('punkt')
nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JBARCO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JBARCO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

non_words = list(punctuation)
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))

def lexeme_tokenize(text):
    text=text.lower()
    doc = nlp(text)
    lemmas = [t.lemma_.lower() for t in doc if not t.is_punct | t.is_stop]
    words = [t.lower() for t in lemmas if len(t) > 3 and t.isalpha()]
    return words 

def stem_tokenize(text, stemmer):
    text = ''.join([c for c in text if c not in non_words])
    tokens=word_tokenize(text)
    stemmed = []
    for item in tokens:
        if item not in spanish_stopwords:
            stemmed.append(stemmer.stem(item))
    return stemmed





modelo_tf_idf = TfidfVectorizer(max_df=0.7, min_df=0.1,
                     lowercase=True,
                     ngram_range = (1,1),
                     analyzer = 'word',
                     tokenizer = lexeme_tokenize,              
                     max_features=7000) 

In [9]:
matrix_tf = modelo_tf_idf.fit_transform(df['resumen'].apply(lambda x: np.str_(x)))

In [10]:
df_estructurado_tf_idf=pd.DataFrame(matrix_tf.toarray(),columns=modelo_tf_idf.get_feature_names())
df_estructurado_tf_idf.to_csv('.\\Datos\\tesis_estructurada_7000.csv',sep='|')

In [11]:
df_estructurado_tf_idf.head()

,abajar,abandonar,abarcar,abastecer,abastecimiento,abonar,abordar,abril,abrir,absoluto,...,étnico,éxito,índice,índole,ítem,óptico,órgano,únicamente,único,útil
0,0.0,0.001839,0.013028,0.0,0.0,0.0,0.000000,0.003424,0.004945,0.003869,...,0.117695,0.001511,0.0,0.006618,0.0,0.004819,0.016193,0.001410,0.005909,0.002907
1,0.0,0.006691,0.009478,0.0,0.0,0.0,0.010570,0.003737,0.000900,0.005630,...,0.000000,0.000000,0.0,0.001605,0.0,0.003506,0.008836,0.010262,0.008599,0.001058
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.002665,0.002827,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.003342,0.000000,0.011706,0.000000
3,0.0,0.000000,0.003197,0.0,0.0,0.0,0.000000,0.013445,0.002427,0.015193,...,0.000000,0.000000,0.0,0.008661,0.0,0.000000,0.007947,0.000000,0.006960,0.000000
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.003129,0.013277,0.002397,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.008203,0.006873,0.000000
